<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/image_creation_tool_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [2]:
!pip install -q -U crewai
!pip install -q -U duckduckgo-search
!pip install -q -U crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.6/324.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 

In [15]:
!pip install crewai[tools]

.7.1->crewai[tools]) (43.0.3)


In [3]:
import os

from crewai import Agent, Task, Crew, Process, LLM
#from crewai_tools import tool
from crewai.tools import BaseTool

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from dotenv import load_dotenv
load_dotenv("/content/drive/MyDrive/collab/.env")

True

In [7]:
from crewai import Agent, Task, Crew
#from crewai_tools import tool
import google.generativeai as genai
import os
import base64
import requests
from PIL import Image
import io

/usr/local/lib/python3.11/dist-packages/google/colab/html/_background_server.py:103: DeprecationWarning: make_current is deprecated; start the event loop first
  ioloop.make_current()


In [8]:
# Configure Gemini API
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [62]:
image_model_name="gemini-2.0-flash-preview-image-generation"
gemini_model="gemini/gemini-2.5-flash"

In [139]:
class ImageGenerator:
    def __init__(self, model_name=image_model_name):
        self.model = genai.GenerativeModel(model_name)

    def generate_image(self, prompt: str, output_path="generated_images"):
        """Generate image using Gemini Flash API and save locally"""
        try:
            # Call Gemini API
            response = self.model.generate_content(
                f"Generate a children's book illustration showing: {prompt}",
                generation_config={"response_mime_type": "image/jpeg"}
            )

            # Extract and save image
            os.makedirs(output_path, exist_ok=True)
            image_data = base64.b64decode(response.candidates[0].content.parts[0].data)
            filename = f"{output_path}/{hash(prompt)}.jpg"

            with open(filename, "wb") as f:
                f.write(image_data)

            return filename

        except Exception as e:
            print(f"Error generating image: {e}")
            return None

In [140]:
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

In [141]:
gemini_image=False

In [180]:
class ImageToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    argument: str = Field(..., description="Description of the argument.")


class ImageTool(BaseTool):
    name: str = "Image creation tool"
    description: str = (
        "use this tool to create an image based on text description as input"
    )
    args_schema: Type[BaseModel] = ImageToolInput

    def _run(self, argument: str) -> str:
      client = genai.Client()
      contents = argument

      response = client.models.generate_content(
          model="gemini-2.0-flash-preview-image-generation",
          contents=contents,
          config=types.GenerateContentConfig(
            response_modalities=['TEXT', 'IMAGE']
            )
        )

      if response.candidates:
          for part in response.candidates[0].content.parts:
            if part.text is not None:
              print(part.text)
            if part.inline_data is not None:
              image = Image.open(BytesIO((part.inline_data.data)))
              image.save('gemini-native-image.png')
              print(os.getcwd())

          print("✅ Image generated successfully!")
      else:
          print("❌ Failed to generate image")
      return BytesIO((part.inline_data.data))



In [181]:
ImageTool()._run("a rabit")

A fluffy brown rabbit with long ears, twitching its nose, sitting on a patch of green grass dotted with small white wildflowers in soft, natural daylight.


/content
✅ Image generated successfully!


In [182]:
# Create Artist Agent
artist_agent = Agent(
    role="Children's Book Illustrator",
    goal="Create engaging, colorful illustrations for children's stories",
    backstory=(
        "A talented digital artist specializing in children's book illustrations. "
        "Known for vibrant colors, expressive characters, and whimsical scenes."
    ),
    tools=[ImageTool()],
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    max_retry_limit=3,
    llm=gemini_model
)

In [183]:
# Create SINGLE image generation task
image_task = Task(
    description=(
        "Generate an illustration for the scene: "
        "'A clever rabbit leading a mighty lion through the jungle towards a well'. "
        "Create a colorful, engaging image suitable for children aged 4-8."
    ),
    agent=artist_agent,
    expected_output="File path to generated JPEG image",
    output_file="lion_rabbit_scene.jpg"
)

In [184]:
# Scene descriptions from previous story generation
scenes = [
        "A mighty lion roaring in the jungle making other animals tremble",
        "Animals gathering to discuss sending daily offerings to the lion",
        "A clever rabbit hatching a plan near a well",
        "Rabbit leading an impatient lion through the jungle",
        "Lion jumping into a well after seeing its reflection"
    ]

In [185]:
# Create crew with SINGLE task
illustration_crew = Crew(
    agents=[artist_agent],
    tasks=[image_task],  # Must be a list even with one task
    verbose=True
)

In [186]:
result = illustration_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ebe3fc25-2ec9-44e0-b785-2609d96fd2a8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Children's Book Illustrator

## Task: Generate an illustration for the scene: 'A clever rabbit leading a mighty lion through the jungle towards 
a well'. Create a colorful, engaging image suitable for children aged 4-8.

Output()

I will generate a vibrant and whimsical illustration of a small brown rabbit winking and looking back at a large, 
friendly lion that is following it with a slightly puzzled but trusting expression. They are walking through a 
lush, colorful jungle filled with exaggerated, happy trees, bright flowers, and exotic plants. A stone well with a 
wooden bucket, partially hidden by foliage, is in the background. The style will be cheerful with soft edges and a 
dreamlike quality, using bright primary and secondary colors.

...

ERROR:root:LiteLLM call failed: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}




 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 2c6facfc-477d-4b39-b116-fa8cf6c0b005                                                                     │
│  Agent: Children's Book Illustrator                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: ebe3fc25-2ec9-44e0-b785-2609d96fd2a8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

InternalServerError: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}


In [172]:
# Minimal task
test_task = Task(
    description="Generate image: 'A simple test circle'",
    agent=artist_agent,
    expected_output="Image file path",
)

# Minimal crew
test_crew = Crew(
    agents=[artist_agent],
    tasks=[test_task],
    verbose=True
)

In [173]:
# Run with guard
try:
    result = test_crew.kickoff()
    print(f"Test result: {result}")
except RecursionError:
    print("Recursion detected - check tool implementation")
    import traceback
    traceback.print_exc()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c0a81410-7144-4727-823e-fb343ce1ac04                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Children's Book Illustrator

## Task: Generate image: 'A simple test circle'

Output()

A minimalist digital illustration of a perfect, unfilled white circle centered on a plain, light gray background.

...

 Error details: maximum recursion depth exceeded while calling a Python object

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 0898404e-0887-4da8-ac9a-7eca87bc6a5e                                                                     │
│  Agent: Children's Book Illustrator                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: c0a81410-7144-4727-823e-fb343ce1ac04                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Recursion detected - check tool implementation


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/formatters.py", line 224, in catch_format_error
    r = method(self, *args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/formatters.py", line 909, in __call__
    printer = self.lookup(obj)
              ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/formatters.py", line 392, in lookup
    if obj_id in self.singleton_printers:
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/traitlets/traitlets.py", line 700, in __get__
    return self.get(obj, cls)
           ^^^^^^^^^^^^^^^^^^
RecursionError: maximum recursion depth exceeded

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipython-input-173-1101025522.py", line 3, in <cell line: 0>
    result = test_crew.kickoff()
             ^^^^^^^^^^^^

In [136]:
class GeminiImageTool(BaseTool):
    # Add type annotations with Field declarations
    name: str = Field(
        default="Gemini Image Generator",
        description="Name of the tool"
    )
    description: str = Field(
        default="Generates images using Gemini Flash model based on text prompts",
        description="Tool description"
    )

    def _run(self, prompt: str, retries: int = 3) -> str:
        """Core image generation logic with enhanced error handling"""
        for attempt in range(retries):
            try:
                #logger.info(f"Generating image for prompt: {prompt[:50]}... (Attempt {attempt+1}/{retries})")

                # Initialize model (fresh instance each try)
                model = genai.GenerativeModel(image_model_name)

                # Generate content
                response = model.generate_content(
                    f"Create a colorful children's book illustration showing: {prompt}",
                    generation_config={
                        "response_mime_type": "image/jpeg",
                        "temperature": 0.7
                    }
                )

                # Validate response
                if not response.candidates:
                    #logger.error("No candidates in response")
                    raise ValueError("No generation candidates returned")

                if not response.candidates[0].content.parts:
                    #logger.error("No content parts in response")
                    raise ValueError("No content parts in response")

                image_data = response.candidates[0].content.parts[0].data
                if not image_data:
                    #logger.error("Empty image data received")
                    raise ValueError("Empty image data")

                # Create output directory
                os.makedirs("generated_images", exist_ok=True)
                filename = f"generated_images/image_{int(time.time())}.jpg"

                # Save image
                with open(filename, "wb") as f:
                    f.write(base64.b64decode(image_data))

                return filename

            except genai.types.BlockedPromptException as e:
                #logger.error(f"Prompt blocked: {e}")
                return f"ERROR: Prompt blocked by safety filters - try a different description"

            except Exception as e:
                #logger.error(f"Attempt {attempt+1} failed: {str(e)}")
                if attempt < retries - 1:
                    wait_time = 2 ** attempt  # Exponential backoff
                    #logger.info(f"Retrying in {wait_time} seconds...")
                else:
                    return f"ERROR: Failed after {retries} attempts: {str(e)}"

        return "ERROR: Unexpected failure in image generation"



In [137]:
# Test function with proper error handling
def test_gemini_tool():
    """Test the image generation tool with various scenarios"""
    test_cases = [
        ("Simple geometric shapes: red circle, blue square", "simple"),
        ("A cartoon rabbit holding a giant carrot", "rabbit"),
        ("Complex scene: jungle with animals and waterfall", "complex"),
        ("", "empty"),  # Test empty prompt
        ("Violent content: fighting animals", "safety")  # Test safety filters
    ]

    tool = GeminiImageTool()

    for prompt, test_name in test_cases:
        result = tool._run(prompt)
        print(f"📝 Result: {result}")




In [138]:
test_gemini_tool()


📝 Result: ERROR: Failed after 3 attempts: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-preview-image-generation:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.generation_config.response_mime_type: allowed mimetypes are `text/plain`, `application/json`, `application/xml`, `application/yaml` and `text/x.enum`.



📝 Result: ERROR: Failed after 3 attempts: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-preview-image-generation:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.generation_config.response_mime_type: allowed mimetypes are `text/plain`, `application/json`, `application/xml`, `application/yaml` and `text/x.enum`.



📝 Result: ERROR: Failed after 3 attempts: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-preview-image-generation:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.generation_config.response_mime_type: allowed mimetypes are `text/plain`, `application/json`, `application/xml`, `application/yaml` and `text/x.enum`.



📝 Result: ERROR: Failed after 3 attempts: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-preview-image-generation:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.generation_config.response_mime_type: allowed mimetypes are `text/plain`, `application/json`, `application/xml`, `application/yaml` and `text/x.enum`.



📝 Result: ERROR: Failed after 3 attempts: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-preview-image-generation:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.generation_config.response_mime_type: allowed mimetypes are `text/plain`, `application/json`, `application/xml`, `application/yaml` and `text/x.enum`.



In [160]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64

# Test direct API call
def test_direct_image_gen():
    client = genai.Client()
    contents = "A simple red circle on white background"

    response = client.models.generate_content(
        model="gemini-2.0-flash-preview-image-generation",
        contents=contents,
        config=types.GenerateContentConfig(
          response_modalities=['TEXT', 'IMAGE']
          )
      )

    if response.candidates:
        for part in response.candidates[0].content.parts:
          if part.text is not None:
            print(part.text)
          if part.inline_data is not None:
            image = Image.open(BytesIO((part.inline_data.data)))
            image.save('gemini-native-image.png')
            print(os.getcwd())

        print("✅ Image generated successfully!")
    else:
        print("❌ Failed to generate image")

In [161]:
test_direct_image_gen()

A perfectly smooth, vibrant red circle centered against a stark white background, with no shadows or textures to distract from its pure geometric form.


/content
✅ Image generated successfully!
